In [1]:
!pip install -U langchain
!pip install avahiplatform -U
!pip install -U langchain-community
!pip install langchain_aws
!pip install langchain_experimental

  Using cached langchain-0.3.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_text_splitters-0.3.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached tenacity-8.5.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached packaging-24.1-py3-none-any.whl.metadata (3.2 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
Using cached langchain-0.3.2-py3-none-any.whl (1.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 20.7 MB/s eta 0:00:00
Using cached langchain_text_splitters-0.3.0-py3-none-any.whl (25 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 29.2 MB/s eta 0:00:00
Using cached tenacity-8.5.0-py3-none-any.whl (28 kB)
Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
Using cached packaging-24.1-py3-none-any.whl (53 kB)
Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl (54 kB)
  Attempting uninstall: tenacity
    Found existing installati

In [2]:
from langchain_aws import ChatBedrock
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
import time
import avahiplatform
import pandas as pd
from langchain.agents import create_sql_agent
from langchain.agents.agent_types import AgentType
from langchain.sql_database import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from dotenv import load_dotenv
import os

In [3]:
load_dotenv("sdk.env")

True

In [5]:
article_1 = """"**Apple Camp returns with free sessions for kids aged 6 to 10 years old**
*By Rich Demuro*
*(KTLA) -- Apple Camp is back in session, offering free workshops at Apple stores across the country. The theme this year: Exploring new worlds and telling stories inspired by kindness.*
*""It taught me how to do stuff that I never knew how to do,"" said Grace Kinsera, an Apple Creative Pro.*
*The free, 90-minute sessions are designed for children ages 6 to 10 years old. They're held throughout the summer.*
*This particular session focuses on using the iPad to create an interactive storybook.*
*""They're creating animations, they're adding AR shapes, 3D shapes, taking AR photos where they place the 3D shapes in the world around them,"" said Kinsera.*
*Kids learn new skills on familiar devices, and sometimes parents do too.*
*""I'm watching some stuff now that I was like, 'I didn't know that my iPad could do that,'"" said Mili Patel, a parent.*
*There are bigger lessons as well.*
*The apple company is located in California and Mumbai and Pune.*
*""There's always room for more kindness, and to get these kids thinking about it in a thoughtful, creative way early on is wonderful,"" said Kinsera.*
*Apple Camp is part of a larger initiative called Today at Apple, which offers free, hands-on sessions that teach valuable skills and hidden tricks.*
*""We all want our kids to learn about technology, but in a safe way. This is a great opportunity for them to do that,"" said Patel.*
*Sign-up is open now. Sessions run through the end of July. All campers get a free T-shirt, too!*
*My name is John and my mail id is johnwick009@gmail.com*. I live in 132, My Street, Kingston, New York 12401, America. My phone number is 98989898987. And my Social security number is AAA-GG-SSSS.
*To sign up, visit apple.com/today.*
*Copyright 2023 Nexstar Media Inc. All rights reserved. This material may not be published, broadcast, rewritten, or redistributed.*"
"""

article_2 = """
Understanding Cybersecurity Threats in 2024
Cybersecurity has become a paramount concern for businesses globally. As hackers grow more sophisticated, the need for robust cybersecurity strategies is essential. John Doe, an IT specialist at SecureNet Solutions, commented on the evolving nature of threats, stating, "We're seeing more targeted attacks than ever before." His email, john.doe@securenetsolutions.com, is often flooded with phishing attempts that mimic legitimate business communications. One such attack targeted the financial department, requesting sensitive details such as Jane Smith's social security number (SSN: 123-45-6789). These fraudulent emails have become increasingly difficult to detect.

In 2023 alone, it was estimated that over 80% of companies experienced some form of cyberattack. The CEO of CyberTech Corp, Michael Thompson, mentioned in a recent interview that "organizations must invest in advanced encryption technologies to safeguard their data." Michael can be reached for further insights at michael.thompson@cybertech.com. With a growing number of ransomware cases, protecting sensitive data like social security numbers, banking information, and personal identification has never been more critical.
"""

article_3 = """
The Role of Artificial Intelligence in Healthcare
Artificial Intelligence (AI) is revolutionizing the healthcare industry, from diagnostics to patient care. Dr. Sarah Lee, a leading AI researcher at MediTech Innovations, has been instrumental in developing AI tools that help doctors make faster, more accurate diagnoses. "AI has the potential to change the way we treat diseases like cancer," she said in a recent webinar. Sarah’s contact, sarah.lee@meditechinnovations.com, is available for further inquiries about the project.

An exciting development from MediTech is a new AI-powered tool that can analyze medical records and flag potential health risks. This tool has helped Robert Miller, a patient who had an undiagnosed heart condition, receive timely treatment. "If it wasn't for this technology, I wouldn't be here today," Robert mentioned. His patient ID (Patient ID: RM2023H567) was flagged during a routine check-up. Privacy and patient confidentiality remain a top priority, with all sensitive data encrypted and stored securely.
"""

article_4 = """
The Growing Importance of Data Privacy
Data privacy has become one of the most critical issues in the digital age. With the introduction of stricter regulations like the General Data Protection Regulation (GDPR), companies are under increasing pressure to protect user information. Anna Roberts, a data privacy consultant at SecureData Solutions, noted that "businesses must be transparent about how they handle customer data." For more information, Anna can be reached at anna.roberts@securedata.com.

One case involved client ID 1459, where a data breach led to the exposure of David Harris's personal information, including his credit card number (CC: 1234 5678 9012 3456). The breach highlighted the vulnerabilities in data storage systems and the need for regular security audits. David's situation is not unique, as over 5 million records were compromised last year in similar attacks.
"""

article_5 = """
The Future of Electric Vehicles (EVs)
As the world moves towards sustainability, electric vehicles (EVs) are becoming more popular. Tom Jenkins, an automotive engineer at GreenDrive Motors, shared his thoughts on the future of EVs, stating, "We are on the brink of a transportation revolution." You can contact Tom at tom.jenkins@greendrive.com for more details on their upcoming projects.

One recent buyer, Emily Parker, registered her vehicle under Vehicle Registration Number: EV456789. She commented, "Switching to an electric vehicle has been a game-changer for me." Emily's experience highlights the growing consumer demand for eco-friendly transportation options. However, the industry still faces challenges such as battery longevity and charging infrastructure.
"""

article_6 = """
The Evolution of E-commerce
E-commerce has transformed the way we shop, especially in the post-pandemic era. Mark Wilson, the CEO of ShopEase, a popular online marketplace, noted that "e-commerce will continue to grow as more consumers embrace online shopping." Mark can be contacted for media inquiries at mark.wilson@shopease.com.

One of ShopEase’s regular customers, Customer ID: 456123, Lily Watson, recently shared her experience shopping online: "It's convenient, but there are still concerns about data security." Lily had a scare when a phishing email requested her billing information, falsely claiming it was from ShopEase. These incidents have raised awareness about online fraud and the importance of secure payment systems.
"""

article_7 = """
The Benefits of Cloud Computing
Cloud computing is becoming the backbone of modern IT infrastructures. Chris Evans, a cloud solutions architect at CloudSync, said, "The cloud has revolutionized the way businesses store and access data." Chris can be contacted at chris.evans@cloudsync.com for more insights on cloud migration strategies.

Recently, Project ID CS7890, led by Amy Johnson, moved their entire IT infrastructure to the cloud, saving them significant costs. "We were initially concerned about data security," said Amy, "but after implementing encryption and multi-factor authentication, we feel more secure than ever." Amy's team ensured that sensitive customer data, including John Peterson's personal records (SSN: 987-65-4321), were securely stored on encrypted cloud servers.
"""

article_8 = """
The Rise of Cryptocurrency
Cryptocurrency continues to gain momentum as an alternative financial system. Daniel Carter, a blockchain developer at CryptoBase, noted, "We are seeing more mainstream adoption of cryptocurrencies like Bitcoin and Ethereum." Daniel can be reached at daniel.carter@cryptobase.com for more insights.

Recently, CryptoWallet ID: BTC987654, owned by Rachel Adams, saw a significant increase in value. Rachel shared, "Investing in cryptocurrency has been both exciting and nerve-wracking. The volatility is real, but the potential returns are worth it." As more people like Rachel invest in cryptocurrencies, ensuring the security of digital wallets and transactions remains a top priority.
"""

article_9 = """
Advancements in Renewable Energy
The renewable energy sector is seeing unprecedented growth, with innovations in solar, wind, and geothermal energy. Laura King, a renewable energy specialist at EcoPower Solutions, explained, "We are on the verge of a green energy revolution." Laura’s contact, laura.king@ecopowersolutions.com, is available for any renewable energy-related queries.

EcoPower recently installed a solar energy system for Client ID: SOL12345, George Williams, who commented, "Switching to solar energy has reduced my electricity bills by 60%." George's installation is part of a larger trend of individuals and businesses adopting clean energy solutions to reduce their carbon footprint.
"""

article_10 = """
The Future of Smart Homes
Smart homes are no longer a futuristic concept; they are becoming a reality. James Turner, a smart home technology consultant at HomeTech Innovations, believes that "the integration of smart devices will redefine the way we live." James is reachable at james.turner@hometech.com for inquiries on smart home installations.

Susan Brooks, who recently installed a complete smart home system under HomeTech ID: HT99876, mentioned, "It’s incredible how much control I have over my home—from security to lighting, all from my phone." Susan’s home is now equipped with the latest in smart security systems, ensuring that her sensitive personal data, including her home address (123 Main St., Anytown, USA), remains protected with top-tier encryption.
"""

In [7]:
all_articles_list = [article_1, article_2, article_3, article_4, article_5,
                     article_6, article_7, article_8, article_9, article_10]

# Summarization

In [9]:
total_time = 0
all_response_list = []

for text in all_articles_list:
    start_time = time.time()
    model_kwargs_claude = {"temperature": 0, "top_p": 0.2, "max_tokens": 1000}
    llm = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0", model_kwargs=model_kwargs_claude)
    prompt = f"""Please summarize the following text: {text}"""
    response = llm.invoke(prompt)
    all_response_list.append(response)
    time_taken = time.time() - start_time
    total_time += time_taken
    # print(response)
average_time = total_time / len(all_articles_list)

print(f"Total time taken by langchain-aws for 10 articles: {total_time}")
print(f"Avaerage time taken by langchain-aws for 10 articles: {average_time}")

5.795977210998535
Total time taken by langchain-aws for 10 articles: 57.95977210998535
Avaerage time taken by langchain-aws for 10 articles: 5.795977210998535


In [11]:
total_time = 0
all_response_list = []

for text in all_articles_list:
    start_time = time.time()
    response, input_tokens, output_tokens, cost = avahiplatform.summarize(text)
    all_response_list.append(response)
    time_taken = time.time() - start_time
    total_time += time_taken
average_time = total_time / len(all_articles_list)

print(f"Total time taken by avahiplatform for 10 articles: {total_time}")
print(f"Avaerage time taken by avahiplatform for 10 articles: {average_time}")

2024-10-08 10:44:24.790 | INFO     | avahiplatform.summarizer:_get_bedrock_client:31 - No explicit credentials provided. Attempting to use default credentials.
2024-10-08 10:44:24.798 | INFO     | avahiplatform.summarizer:_get_model_details:112 - Using model: anthropic.claude-3-sonnet-20240229-v1:0
2024-10-08 10:44:24.799 | INFO     | avahiplatform.summarizer:model_invoke:62 - Invoking model to summarize content.
2024-10-08 10:44:33.374 | INFO     | avahiplatform.summarizer:model_invoke:80 - Model invocation successful. Total cost: $0.004548
2024-10-08 10:44:33.382 | INFO     | avahiplatform.summarizer:_get_bedrock_client:31 - No explicit credentials provided. Attempting to use default credentials.
2024-10-08 10:44:33.387 | INFO     | avahiplatform.summarizer:_get_model_details:112 - Using model: anthropic.claude-3-sonnet-20240229-v1:0
2024-10-08 10:44:33.389 | INFO     | avahiplatform.summarizer:model_invoke:62 - Invoking model to summarize content.
2024-10-08 10:44:42.646 | INFO     

Total time taken by avahiplatform for 10 articles: 63.60561656951904
Avaerage time taken by avahiplatform for 10 articles: 6.360561656951904


# Structured information extraction

In [12]:
total_time = 0
all_response_list = []

for text in all_articles_list:
    start_time = time.time()
    model_kwargs_claude = {"temperature": 0, "top_p": 0.2, "max_tokens": 1000}
    llm = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0", model_kwargs=model_kwargs_claude)
    prompt = f"""extract the below entities from this model in structured format:
                1. Name:
                2. Places:
                Text: {text}
                response should be json in below format
                """ + """response should be json in below format
                {"Name": ["Name1", "Name2"],"Places": ["Place1","Place2"]}
                Make sure output has only json output. No other extra words"""
    response = llm.invoke(prompt)
    all_response_list.append(response)
    time_taken = time.time() - start_time
    total_time += time_taken
    # print(response)
average_time = total_time / len(all_articles_list)

print(f"Total time taken by langchain-aws for 10 articles: {total_time}")
print(f"Avaerage time taken by langchain-aws for 10 articles: {average_time}")

Total time taken by langchain-aws for 10 articles: 14.038135528564453
Avaerage time taken by langchain-aws for 10 articles: 1.4038135528564453


In [14]:
total_time = 0
all_response_list = []

for text in all_articles_list:
    start_time = time.time()
    response, input_tokens, output_tokens, cost = avahiplatform.structredExtraction(text)
    all_response_list.append(response)
    time_taken = time.time() - start_time
    total_time += time_taken
average_time = total_time / len(all_articles_list)

print(f"Total time taken by avahiplatform for 10 articles: {total_time}")
print(f"Avaerage time taken by avahiplatform for 10 articles: {average_time}")

2024-10-08 10:48:27.247 | INFO     | avahiplatform.structredExtraction:_get_bedrock_client:31 - No explicit credentials provided. Attempting to use default credentials.
2024-10-08 10:48:27.253 | INFO     | avahiplatform.structredExtraction:_get_model_details:112 - Using model: anthropic.claude-3-sonnet-20240229-v1:0
2024-10-08 10:48:27.255 | INFO     | avahiplatform.structredExtraction:model_invoke:62 - Invoking model to extracting content.
2024-10-08 10:48:29.929 | INFO     | avahiplatform.structredExtraction:model_invoke:80 - Model invocation successful. Total cost: $0.002838
2024-10-08 10:48:29.933 | INFO     | avahiplatform.structredExtraction:_get_bedrock_client:31 - No explicit credentials provided. Attempting to use default credentials.
2024-10-08 10:48:29.940 | INFO     | avahiplatform.structredExtraction:_get_model_details:112 - Using model: anthropic.claude-3-sonnet-20240229-v1:0
2024-10-08 10:48:29.941 | INFO     | avahiplatform.structredExtraction:model_invoke:62 - Invoking

Total time taken by avahiplatform for 10 articles: 16.280076503753662
Avaerage time taken by avahiplatform for 10 articles: 1.6280076503753662


# Data masking

In [16]:
total_time = 0
all_response_list = []

for text in all_articles_list:
    start_time = time.time()
    model_kwargs_claude = {"temperature": 0, "top_p": 0.2, "max_tokens": 1000}
    llm = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0", model_kwargs=model_kwargs_claude)
    prompt = f"""
        Selectively mask specific types of sensitive data in the following text by replacing only the characters of the sensitive information with '*'. Leave all other text unchanged.

        Sensitive data types to mask:
        1. Names (first and last)
        2. Email addresses
        3. Social Security Numbers (9-digit numbers)
        4. Phone numbers (various formats)

        Text to process:
        {text}

        Instructions:
        1. Identify instances of the above sensitive data types in the text.
        2. Replace each character of ONLY the identified sensitive data with '*'.
        3. Leave all other text completely unchanged.
        4. Return the entire text with only the sensitive data masked.
        5. Do not add any explanations or additional text to your response.

        Example:
        Input: "John Doe's email is john@example.com and SSN is 123-45-6789."
        Output: "**** ***'s email is ***************** and SSN is ***********."

        Processed text:
    """
    response = llm.invoke(prompt)
    all_response_list.append(response)
    time_taken = time.time() - start_time
    total_time += time_taken
    # print(response)
average_time = total_time / len(all_articles_list)

print(f"Total time taken by langchain-aws for 10 articles: {total_time}")
print(f"Avaerage time taken by langchain-aws for 10 articles: {average_time}")

Total time taken by langchain-aws for 10 articles: 57.614434242248535
Avaerage time taken by langchain-aws for 10 articles: 5.761443424224853


In [17]:
total_time = 0
all_response_list = []

for text in all_articles_list:
    start_time = time.time()
    response, input_tokens, output_tokens, cost = avahiplatform.DataMasking(text)
    all_response_list.append(response)
    time_taken = time.time() - start_time
    total_time += time_taken
average_time = total_time / len(all_articles_list)

print(f"Total time taken by avahiplatform for 10 articles: {total_time}")
print(f"Avaerage time taken by avahiplatform for 10 articles: {average_time}")

2024-10-08 10:52:15.807 | INFO     | avahiplatform.data_masking:_get_bedrock_client:31 - No explicit credentials provided. Attempting to use default credentials.
2024-10-08 10:52:15.817 | INFO     | avahiplatform.data_masking:_get_model_details:112 - Using model: anthropic.claude-3-sonnet-20240229-v1:0
2024-10-08 10:52:15.818 | INFO     | avahiplatform.data_masking:model_invoke:62 - Invoking model to extracting content.
2024-10-08 10:52:28.772 | INFO     | avahiplatform.data_masking:model_invoke:80 - Model invocation successful. Total cost: $0.009882
2024-10-08 10:52:28.777 | INFO     | avahiplatform.data_masking:_get_bedrock_client:31 - No explicit credentials provided. Attempting to use default credentials.
2024-10-08 10:52:28.784 | INFO     | avahiplatform.data_masking:_get_model_details:112 - Using model: anthropic.claude-3-sonnet-20240229-v1:0
2024-10-08 10:52:28.785 | INFO     | avahiplatform.data_masking:model_invoke:62 - Invoking model to extracting content.
2024-10-08 10:52:36

Total time taken by avahiplatform for 10 articles: 65.15433835983276
Avaerage time taken by avahiplatform for 10 articles: 6.515433835983276


# Grammar correction

In [18]:
total_time = 0
all_response_list = []

for text in all_articles_list:
    start_time = time.time()
    model_kwargs_claude = {"temperature": 0, "top_p": 0.2, "max_tokens": 1000}
    llm = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0", model_kwargs=model_kwargs_claude)
    prompt = f"""
        Please rectify grammar and spelling of the following text: {text}
    """
    response = llm.invoke(prompt)
    all_response_list.append(response)
    time_taken = time.time() - start_time
    total_time += time_taken
    # print(response)
average_time = total_time / len(all_articles_list)

print(f"Total time taken by langchain-aws for 10 articles: {total_time}")
print(f"Avaerage time taken by langchain-aws for 10 articles: {average_time}")

Total time taken by langchain-aws for 10 articles: 48.86798810958862
Avaerage time taken by langchain-aws for 10 articles: 4.8867988109588625


In [19]:
total_time = 0
all_response_list = []

for text in all_articles_list:
    start_time = time.time()
    response, input_tokens, output_tokens, cost = avahiplatform.grammarAssistant(text)
    all_response_list.append(response)
    time_taken = time.time() - start_time
    total_time += time_taken
average_time = total_time / len(all_articles_list)

print(f"Total time taken by avahiplatform for 10 articles: {total_time}")
print(f"Avaerage time taken by avahiplatform for 10 articles: {average_time}")

2024-10-08 10:57:06.418 | INFO     | avahiplatform.grammarCorrection:_get_bedrock_client:31 - No explicit credentials provided. Attempting to use default credentials.
2024-10-08 10:57:06.423 | INFO     | avahiplatform.grammarCorrection:_get_model_details:112 - Using model: anthropic.claude-3-sonnet-20240229-v1:0
2024-10-08 10:57:06.425 | INFO     | avahiplatform.grammarCorrection:model_invoke:62 - Invoking model to rectify grammar and spelling of the content.
2024-10-08 10:57:19.737 | INFO     | avahiplatform.grammarCorrection:model_invoke:80 - Model invocation successful. Total cost: $0.009615
2024-10-08 10:57:19.741 | INFO     | avahiplatform.grammarCorrection:_get_bedrock_client:31 - No explicit credentials provided. Attempting to use default credentials.
2024-10-08 10:57:19.745 | INFO     | avahiplatform.grammarCorrection:_get_model_details:112 - Using model: anthropic.claude-3-sonnet-20240229-v1:0
2024-10-08 10:57:19.747 | INFO     | avahiplatform.grammarCorrection:model_invoke:62

Total time taken by avahiplatform for 10 articles: 57.456849575042725
Avaerage time taken by avahiplatform for 10 articles: 5.745684957504272


# CSV querying

In [20]:
start_time = time.time()
query = "What are the average hours studies when parental involvement is low?"
model_kwargs_claude = {"temperature": 0, "top_p": 0.2, "max_tokens": 250}
llm = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0", model_kwargs=model_kwargs_claude)
df = pd.read_csv("./StudentPerformanceFactors.csv")
agent = create_pandas_dataframe_agent(
            llm,
            df,
            verbose=False,
            agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
            allow_dangerous_code=True
            )

agent_response = agent.invoke(query)['output']
total_time = time.time() - start_time
print(agent_response)
print(f"Total time taken by langchain-aws: {total_time}")

The average hours studied when parental involvement is low is 20.13462976813762.
Total time taken by langchain-aws: 7.44120192527771


In [21]:
start_time = time.time()
user_query = "What are the average hours studies when parental involvement is low?"
csv_files = {
    "df1": "./StudentPerformanceFactors.csv"
}
csv_file_path = "./StudentPerformanceFactors.csv"
csv_answer = avahiplatform.query_csv(user_query, csv_files)
total_time = time.time() - start_time
print(csv_answer)
print(f"Total time taken by avahiplatform: {total_time}")

2024-10-08 11:07:51.705 | INFO     | avahiplatform.query_csv:_get_bedrock_client:86 - No explicit credentials provided. Attempting to use default credentials.
2024-10-08 11:07:51.739 | INFO     | avahiplatform.query_csv:_get_model_details:226 - Using model: anthropic.claude-3-sonnet-20240229-v1:0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6607 entries, 0 to 6606
Columns: 20 entries, Hours_Studied to Exam_Score
dtypes: int64(7), object(13)
memory usage: 1.0+ MB


2024-10-08 11:07:54.120 | INFO     | avahiplatform.query_csv:_execute_query:165 - assistant_message: df1.loc[df1['Parental_Involvement'] == 'Low', 'Hours_Studied'].mean()
2024-10-08 11:07:54.122 | INFO     | avahiplatform.query_csv:model_invoke:104 - query output: 20.13462976813762


The average hours studied when parental involvement is low is 20.13462976813762.
Total time taken by avahiplatform: 2.9465699195861816


# NL2SQL

In [22]:
db_type=os.getenv("db_type")
username_db=os.getenv("username_db")
password=os.getenv("password")
host=os.getenv("host")
dbname=os.getenv("dbname")

In [23]:
start_time = time.time()
# Setup database
db = SQLDatabase.from_uri(
    f"postgresql+psycopg2://{username_db}:{password}@{host}:5432/{dbname}",
)
# question = "what this table is about?"
model_kwargs_claude = {"temperature": 0, "top_p": 0.2, "max_tokens": 250}
llm = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0", model_kwargs=model_kwargs_claude)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_executor = create_sql_agent(
llm=llm,
toolkit=toolkit,
verbose=False,
agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)
answer = agent_executor.invoke("what this table is about?")["output"]
print(answer)
total_time = time.time() - start_time
print(f"Total time taken by langchain-aws: {total_time}")

This database is used for managing user accounts, defining permissions, and potentially issuing API keys to users based on their granted permissions. It allows tracking which users belong to which teams, have admin privileges, and have been granted specific permissions and API keys for accessing tools or services.
Total time taken by langchain-aws: 26.77446722984314


In [24]:
start_time = time.time()
nl2sql_result = avahiplatform.nl2sql("what this table is about?",
                                      db_type=db_type, username=username_db, password=password,
                                      host=host, port=5432, dbname=dbname)
print("NL2SQL Result:", nl2sql_result)
total_time = time.time() - start_time
print(f"Total time taken by avahiplatform: {total_time}")

2024-10-08 11:09:54.943 | INFO     | avahiplatform.nl2sql:_get_bedrock_client:30 - No explicit credentials provided. Attempting to use default credentials.
2024-10-08 11:09:54.951 | INFO     | avahiplatform.nl2sql:_get_model_details:158 - Using model: anthropic.claude-3-sonnet-20240229-v1:0


NL2SQL Result: The database schema you provided consists of four tables:

1. **users**: This table stores information about users, including their user_id, username, password, name, email, team, admin status, archive status, and timestamps for when the record was created and last updated.

2. **apikeys**: This table stores API keys associated with users. It contains columns for the api_key_id, user_id (linking it to the users table), openai_api_key, bedrock_api_key, archive status, and timestamps for creation and last update.

3. **userpermissions**: This table acts as a junction table between users and permissions. It contains columns for the user_permission_id, user_id (linking it to the users table), and permission_id (linking it to the permissions table).

4. **permissions**: This table stores information about different permissions. It has columns for the permission_id and permission_name.

The schema suggests that this database is used to manage user accounts, their associated AP